In [6]:
import pandas as pd
import numpy as np

import os

# Create a dictionary to store all dataframes 
path = os.getcwd()
dataset = {}
for file_name in os.listdir(path):
    if file_name.endswith('.parquet.gzip'):
        name = (file_name).split('.')[0]
        dataset[name] = pd.read_parquet(file_name)
import pgeocode

nomi = pgeocode.Nominatim('be')

aed = dataset['aed_locations'].copy()
aed.dropna(subset = ['postal_code'],inplace=True)
aed.index = range(len(aed))
aed['postal_code'] = [round(i) for i in aed['postal_code']]
aed['postal_code'] = aed['postal_code'].astype('str')
aed['coord_x'] = (nomi.query_postal_code(aed['postal_code'].tolist()).latitude)
aed['coord_y'] = (nomi.query_postal_code(aed['postal_code'].tolist()).longitude)
aed.dropna(subset=['coord_x','coord_y'],inplace=True)
aed.index = range(len(aed))

In [12]:
# obtain right gps info for streets
street_names = np.unique(aed[aed['address'].notna()]['address']).tolist()
def chunks(lst, n):
    sublst = []
    for i in range(0, len(lst), n):
        sublst.append(lst[i:i + n])
    return sublst
lst = chunks(street_names,600)
# from geopy.geocoders import Nominatim

# for i in range(len(lst)):
#     sublst = lst[i]
#     geolocator = Nominatim(user_agent="MyApp")
#     locations = []
#     for street in sublst:
#         location = geolocator.geocode(street)
#         if location:
#             locations.append([street,location.latitude,location.longitude])
#     locations = pd.DataFrame(locations)
#     locations = locations.rename(columns={0:"StreetName AED", 1:"Latitude AED", 2:"Longitude AED"})
#     locations.to_csv('StreetInfo_AED'+str(i)+'.csv', index = False)
#     print('complete')

In [14]:
path = os.getcwd()
AED = {}
for file_name in os.listdir(path):
    if file_name.startswith('StreetInfo_AED'):
        name = (file_name).split('.')[0]
        AED[name] = pd.read_csv(file_name)

In [22]:
StreetInfoAED = pd.DataFrame()
for key, value in AED.items():
    StreetInfoAED = pd.concat([StreetInfoAED,value])
StreetInfoAED.index = range(len(StreetInfoAED))
StreetInfoAED.rename(columns={'StreetName AED':'address'},inplace=True)

In [89]:
df = aed.merge(StreetInfoAED,on='address',how='left')
for i in range(len(df)):
    latitude = df.loc[i,'Latitude AED']
    longitude = df.loc[i,'Longitude AED']
    coord_x =  df.loc[i,'coord_x']
    coord_y = df.loc[i,'coord_y']
    if pd.isna(df.loc[i,'Latitude AED']):
        df.loc[i,'Latitude AED'] = coord_x
        df.loc[i,'Longitude AED'] = coord_y
    else:
        if abs(latitude-coord_x)>0.3:
            df.loc[i,'Latitude AED'] = coord_x
        if abs(longitude-coord_y)>0.2:
            df.loc[i,'Longitude AED'] = coord_y
df.to_csv('AED_info.csv',index=False)